In [1]:
import pandas as pd
import requests
import csv

In [ ]:
import requests
import csv
import pandas as pd  
from openpyxl import Workbook  
from dotenv import load_dotenv 
import os  


load_dotenv()


API_KEY = os.getenv("API_KEY")  
if not API_KEY:
    raise ValueError("La clave de API no está configurada. Asegúrate de definir 'API_KEY' en el archivo .env.")

BASE_URL = "https://api.sam.gov/opportunities/v2/search"  


params = {
    "api_key": API_KEY,
    "typeOfSetAside": "SBA",  # Total Small Business Set-Aside
    "postedFrom": "06/01/2025",  # Fecha de inicio en formato MM/dd/yyyy
    "postedTo": "06/10/2025",  # Fecha de fin en formato MM/dd/yyyy
    "limit": "1000",  # Número de resultados por página
    "offset": 0,  # Paginación
    "ptype": "o",  # Tipo de adquisición (solicitud)
}


try:
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()  
    data = response.json()  

    
    results = data.get("opportunitiesData", [])
    if not results:
        print("No se encontraron resultados.")
        exit()

    
    processed_results = []
    for idx, opportunity in enumerate(results, start=1):
        flattened_opportunity = {}

        def flatten_dict(d, parent_key=""):
            """Función recursiva para aplanar un diccionario anidado."""
            for k, v in d.items():
                new_key = f"{parent_key}.{k}" if parent_key else k
                if isinstance(v, dict):
                    flatten_dict(v, new_key)
                else:
                    flattened_opportunity[new_key] = v

        flatten_dict(opportunity)
        processed_results.append(flattened_opportunity)
        print(f"{idx}. Oportunidad procesada con ID: {opportunity.get('noticeId', 'No disponible')}")

    
    csv_file = "samgov_opportunities_full.csv"
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        fieldnames = set()
        for result in processed_results:
            fieldnames.update(result.keys())
        fieldnames = sorted(fieldnames)  # Ordenar campos alfabéticamente

        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(processed_results)

    print(f"Los resultados se han exportado a {csv_file}")

    
    xlsx_file = "samgov.xlsx"
    columns_to_include = [
        "postedDate",
        "solicitationNumber",
        "title",
        "responseDeadLine",
        "fullParentPathName",
        "naicsCode",
        "placeOfPerformance.state.code",
        "placeOfPerformance.state.name",
        "placeOfPerformance.city.name",
        "placeOfPerformance.city.code",
        "placeOfPerformance.zip",
        "placeOfPerformance.streetAddress",
        "uiLink"
    ]

    
    df = pd.read_csv(csv_file)

    
    filtered_df = df[[col for col in columns_to_include if col in df.columns]]

    
    filtered_df.to_excel(xlsx_file, index=False, engine="openpyxl")

    print(f"El archivo {csv_file} se ha convertido exitosamente a {xlsx_file} con las columnas especificadas.")

except requests.exceptions.RequestException as e:
    print(f"Error al realizar la solicitud: {e}")
except FileNotFoundError:
    print(f"El archivo {csv_file} no se encontró. Asegúrate de que exista en el directorio actual.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

1. Oportunidad procesada con ID: fd019218914147f0842ed74c814727cf
2. Oportunidad procesada con ID: f90d136007754e568873e544f4ec0d2e
3. Oportunidad procesada con ID: e511fc1ce5814f8cb05314daa1e86dc9
4. Oportunidad procesada con ID: df38f881177b4f758d8236495511bb79
5. Oportunidad procesada con ID: de25160be58b4d0aad140d64b95d3b9d
6. Oportunidad procesada con ID: cd4c054a9b5743e195998c01ec33c984
7. Oportunidad procesada con ID: cd13059f0a2544d3875bc5c6244564fa
8. Oportunidad procesada con ID: cab9158aef72414b9f268a11dbd552c0
9. Oportunidad procesada con ID: c5a74f73338f460eb571a583d4da6d9c
10. Oportunidad procesada con ID: b688de3028444345825898a1ed5834ab
11. Oportunidad procesada con ID: adc2238398e24a5ca44ee133ff08d608
12. Oportunidad procesada con ID: ab5c8619d0624f6b867185d8194ef459
13. Oportunidad procesada con ID: a74b0244aa4946b99878f89662d67218
14. Oportunidad procesada con ID: 99c20540dec84296912e5380490f49fe
15. Oportunidad procesada con ID: 933bf9d8923247dfb8c95ec6c4395dfc
16. 

In [5]:
import requests
import csv

# Configuration
API_KEY = "K6EJ5ueJMl5zQGXjDwSW6bwPvOUqkT26miUAigla"  # Replace with your API key
BASE_URL = "https://api.sam.gov/opportunities/v2/search"  # Updated endpoint

# Search parameters (all supported by the API)
params = {
    "api_key": API_KEY,
    "typeOfSetAside": "SBA",  # Total Small Business Set-Aside
    "postedFrom": "06/01/2025",  # Start date in MM/dd/yyyy format
    "postedTo": "06/10/2025",  # End date in MM/dd/yyyy format
    "limit": "1000",  # Number of results per page
    "offset": 0,  # Pagination offset
    "ptype": "o",  # Procurement type (e.g., solicitation)
    "solnum": "",  # Solicitation number
    "noticeId": "",  # Notice ID
    "title": "",  # Title of the opportunity
    "state": "",  # State where the opportunity is located
    "zip": "",  # ZIP code of the location
    "organizationName": "",  # Name of the organization
    "typeOfSetAsideDescription": "",  # Set-aside description
    "ncode": "",  # NAICS code
    "ccode": "",  # Classification code
    "rdlfrom": "",  # Response deadline from (MM/dd/yyyy)
    "rdlto": "",  # Response deadline to (MM/dd/yyyy)
}

# Make the request
try:
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()  # Parse the JSON response

    # Process the results
    results = data.get("opportunitiesData", [])
    if not results:
        print("No results found.")
        exit()

    # Dynamically extract all available fields
    processed_results = []
    for idx, opportunity in enumerate(results, start=1):
        # Flatten the opportunity data
        flattened_opportunity = {}

        def flatten_dict(d, parent_key=""):
            """Recursive function to flatten a nested dictionary."""
            for k, v in d.items():
                new_key = f"{parent_key}.{k}" if parent_key else k
                if isinstance(v, dict):
                    flatten_dict(v, new_key)
                else:
                    flattened_opportunity[new_key] = v

        flatten_dict(opportunity)

        # Add the processed opportunity to the list
        processed_results.append(flattened_opportunity)

        # Print to console (optional)
        print(f"{idx}. Processed opportunity with ID: {opportunity.get('noticeId', 'Not available')}")

    # Export to CSV
    csv_file = "samgov_opportunities_full.csv"
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        # Dynamically generate CSV headers
        fieldnames = set()
        for result in processed_results:
            fieldnames.update(result.keys())
        fieldnames = sorted(fieldnames)  # Sort fields alphabetically

        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(processed_results)

    print(f"The results have been exported to {csv_file}")

except requests.exceptions.RequestException as e:
    print(f"Error making the request: {e}")
    if response is not None:
        print(f"Error details: {response.text}")

# Input CSV file name
csv_file = "samgov_opportunities_full.csv"

# Output Excel file name
xlsx_file = "samgov.xlsx"

# Columns to include in the Excel file (in the specified order)
columns_to_include = [
    "postedDate",
    "solicitationNumber",
    "title",
    "responseDeadLine",
    "fullParentPathName",
    "naicsCode",
    "placeOfPerformance.state.code",
    "placeOfPerformance.state.name",
    "placeOfPerformance.city.name",
    "placeOfPerformance.city.code",
    "placeOfPerformance.zip",
    "placeOfPerformance.streetAddress",
    "uiLink"
]

try:
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Filter the DataFrame to include only the specified columns
    # If some columns are missing, they will be ignored
    filtered_df = df[[col for col in columns_to_include if col in df.columns]]

    # Save the filtered DataFrame as an Excel file
    filtered_df.to_excel(xlsx_file, index=False, engine="openpyxl")

    print(f"The file {csv_file} has been successfully converted to {xlsx_file} with the specified columns.")

except FileNotFoundError:
    print(f"The file {csv_file} was not found. Make sure it exists in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")

1. Processed opportunity with ID: fd019218914147f0842ed74c814727cf
2. Processed opportunity with ID: f90d136007754e568873e544f4ec0d2e
3. Processed opportunity with ID: e511fc1ce5814f8cb05314daa1e86dc9
4. Processed opportunity with ID: df38f881177b4f758d8236495511bb79
5. Processed opportunity with ID: de25160be58b4d0aad140d64b95d3b9d
6. Processed opportunity with ID: cd4c054a9b5743e195998c01ec33c984
7. Processed opportunity with ID: cd13059f0a2544d3875bc5c6244564fa
8. Processed opportunity with ID: cab9158aef72414b9f268a11dbd552c0
9. Processed opportunity with ID: c5a74f73338f460eb571a583d4da6d9c
10. Processed opportunity with ID: c3011d40b6ec46a6bc9ca752fbb3255e
11. Processed opportunity with ID: b688de3028444345825898a1ed5834ab
12. Processed opportunity with ID: adc2238398e24a5ca44ee133ff08d608
13. Processed opportunity with ID: ab5c8619d0624f6b867185d8194ef459
14. Processed opportunity with ID: a74b0244aa4946b99878f89662d67218
15. Processed opportunity with ID: 99c20540dec84296912e53